In this notebook, we explore the impact of school holidays. These are also a good indicator when workers take holidays. But we need to be careful what we claim, let's see...

In [ ]:
from data_io.loader.data_loader import DataLoader
from analysis.visualization.characterisation.plotting import plot_hourly_indices_all_subplots, plot_daily_indices_all_subplots
from analysis.visualization.characterisation.plotting import plot_hourly_indices_subplots, plot_daily_indices_subplots
dl = DataLoader()

# Public Holiday vs No Public Holiday

In [ ]:
# This are all public holidays.
intervals = dl.get_all_holiday_intervals(school_vacation=False)

plot_hourly_indices_all_subplots(
    dl,
    filter_dates=intervals,
    title_1="Hourly traffic indices across all stations",
    title_2="Hourly traffic indices across all stations on Public Holidays"
)

plot_daily_indices_all_subplots(
    dl,
    filter_dates=intervals,
    title_1="Daily traffic indices across all stations",
    title_2="Daily traffic indices across all stations on Public Holidays"
)

As expected, public holidays should behave as weekends, but there is more activity which clearly allows the conclusion, that at least some public holidays seem to be
special and enforce more people to go out.

But the weekly plot ???

In [ ]:
for station in dl.get_bicyle_stations():
  plot_hourly_indices_subplots(loader=dl, station_name=station, channel="channels_all", interval=None, ylim=(0, 0.2), show_metrics=False, title_1=None, title_2="(Public Holidays)", filter_dates=intervals)
  # plot_daily_indices_subplots(loader=dl, station_name=station, channel="channels_all", interval=None, ylim=(0, 0.2), title_1=None, title_2="(Public Holidays)", filter_dates=intervals)
  # does not work correctly

On Public Holidays, all working patterns disappear, and some stations, especially Schlierbacher Landstraße show a strong recreational pattern.

# Summer Holidays vs No Summer Holidays

In [ ]:
# This are all summer holidays.
intervals = dl.get_all_holiday_intervals(name="Sommer", school_vacation=True, public_holidays=False)

plot_hourly_indices_all_subplots(
    dl,
    filter_dates=intervals,
    neg_dates=True,
    title_1="Hourly traffic indices across all stations - No Summer Holidays",
    title_2="Hourly traffic indices across all stations - Summer Holidays"
)

plot_daily_indices_all_subplots(
    dl,
    filter_dates=intervals,
    title_1="Daily traffic indices across all stations - No Summer Holidays",
    title_2="Daily traffic indices across all stations - Summer Holidays"
)

During summer holidays where also many working people take their holidays, mid day activity raises, and the morning peaks drops slightly. But the left graph also contains the winter months, interpret with caution.

In [ ]:
for station in dl.get_bicyle_stations():
  plot_hourly_indices_subplots(loader=dl, station_name=station, channel="channels_all", interval=None, ylim=(0, 0.2), show_metrics=False, title_1="(No Summer Holidays)", title_2="(Summer Holidays)", filter_dates=intervals, neg_dates=True)

Rohrbacherstraße looks really strange, if the normalized counts should add up to one, something is off here. But in general, in the Summerholidays there is a drop in the normal working peaks, increased mid day activity, moving all classes closer to mixed.

# Winter Holidays vs No Winter Holidays

In [ ]:
# This are all summer holidays.
intervals = dl.get_all_holiday_intervals(name="Weihnachten", school_vacation=True, public_holidays=False)

plot_hourly_indices_all_subplots(
    dl,
    filter_dates=intervals,
    neg_dates=True,
    title_1="Hourly traffic indices across all stations - No Winter Holidays",
    title_2="Hourly traffic indices across all stations - Winter Holidays"
)

plot_daily_indices_all_subplots(
    dl,
    filter_dates=intervals,
    title_1="Daily traffic indices across all stations - No Winter Holidays",
    title_2="Daily traffic indices across all stations - Winter Holidays"
)

In [ ]:
for station in dl.get_bicyle_stations():
  plot_hourly_indices_subplots(loader=dl, station_name=station, channel="channels_all", interval=None, ylim=(0, 0.2), show_metrics=False, title_1="(No Winter Holidays)", title_2="(Winter Holidays)", filter_dates=intervals, neg_dates=True)

Most people are taking their holidays almost over the complete christmas break.

# What's the most popular public holiday?

In [ ]:
# This are all public holidays.
holidays = {}
intervals = dl.get_all_holiday_intervals(school_vacation=False)

for station in dl.get_bicyle_stations():
    df = dl.get_bicycle(station, sample_rate="1d").filter_intervals(intervals).to_pandas()

    for idx, row in df.iterrows():
        date = row['datetime'].date()
        holiday_info = dl.get_public_holiday(date.strftime("%Y-%m-%d"))
        if len(holiday_info) > 0:
            name = holiday_info['name'][0]
            holidays[name] = holidays.get(name, 0) + row['channels_all']

for name, count in holidays.items():
    print(f"{name}: {count} Radfahrer")

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

names = list(holidays.keys())
counts = np.array([holidays[name] for name in names])

counts_norm = (counts - counts.min()) / (counts.max() - counts.min())

plt.figure(figsize=(12, 6))
bars = plt.bar(names, counts_norm, color='skyblue')
plt.ylabel("Normalized cyclist count")
plt.title("Normalized cyclist counts per public holiday")
plt.xticks(rotation=45, ha='right')

for bar, count in zip(bars, counts_norm):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height(), f"{count:.2f}", 
             ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()